# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [30]:
from datasets import load_dataset

# Nombre del dataset en el hub de Hugging Face
dataset_name = "mrm8488/CHISTES_spanish_jokes"

# Cargar el dataset
dataset = load_dataset(dataset_name, split='train').train_test_split(test_size = 0.2)

# Mostrar un ejemplo del dataset
print(dataset['train'][0])

{'id': 1596, 'text': '- Dígame cuál es su dirección para mandarle los documentos. \n- Calle Bosque, 31. \n- ¿Apartado de Correos? \n- La verdad es que sí, queda un poco lejos.', 'keywords': 'correos', 'funny': 3, 'category': 'otros'}


In [31]:
print(dataset['train'][0:10])  
print(dataset)  


{'id': [1596, 2378, 1978, 123, 171, 1572, 2311, 1888, 1453, 705], 'text': ['- Dígame cuál es su dirección para mandarle los documentos. \n- Calle Bosque, 31. \n- ¿Apartado de Correos? \n- La verdad es que sí, queda un poco lejos.', '-Perdone ¿puede decirme donde está la acera de enfrente?\n-Sí claro. Es aquella de allí.\n-¡No me joda! Pero si vengo de allí y me han dicho que es ésta.', 'Cuatro enfermeras se reúnen en la cafetería del Hospital despuésde haber pasado una noche de sexo en el mismo, y se ponen a comentar:\n- El médico con que me acosté anoche creo que era un anestesista.\n- ¿Por qué?Preguntan las otras\n- Porque no sentí nada, pero nada de nada\n- Pues yo debí acostarme con el Director\n- ¿Por qué?\n- El daba órdenes y todo el trabajo lo tuve que hacer yo \n- Jaaa!!! El mío debió ser un médico Residente. Se pasó todo el rato preguntando, ¿Va todo bien? ¿Lo estoy haciendo bien? ¿Es así cómo se hace?:\n\nLa 4ta Enfermera estaba con una cara de gran felicidad y le preguntan:\

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding


def process_lines(text_input):
    # Tokeniza el texto
    tokenized_text = tokenizer(
        text_input["text"],
        padding="max_length",
        max_length=1024,  # Establece la longitud máxima que tu modelo puede manejar
        truncation=True
    )  
    # Añade la etiqueta 'funny' a las salidas del tokenizer bajo la clave correcta 'labels'
    tokenized_text['labels'] = text_input['funny']-1  # adjust labes to index number starting from 0
    return tokenized_text



##model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

splits=["train","test"]

# Elimina la columna 'id' del dataset 
dataset["train"] = dataset["train"].remove_columns("id")
dataset["train"] = dataset["train"].remove_columns("keywords")
dataset["train"] = dataset["train"].remove_columns("category")

dataset["test"] = dataset["test"].remove_columns("id")
dataset["test"] = dataset["test"].remove_columns("keywords")
dataset["test"] = dataset["test"].remove_columns("category")


tokenized_dataset = {}
for split in dataset.keys():
    # Aplica la función de tokenización y etiquetado a cada split del dataset
    tokenized_dataset[split] = dataset[split].map(process_lines)
    tokenized_dataset[split] = tokenized_dataset[split].remove_columns("funny")

print(tokenized_dataset['train'][0])



Map:   0%|          | 0/1935 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

{'text': '- Dígame cuál es su dirección para mandarle los documentos. \n- Calle Bosque, 31. \n- ¿Apartado de Correos? \n- La verdad es que sí, queda un poco lejos.', 'input_ids': [12, 360, 8836, 6057, 18912, 6557, 75, 1658, 424, 19958, 35764, 18840, 31215, 6855, 283, 293, 22346, 3188, 418, 13, 220, 198, 12, 2199, 293, 14548, 4188, 11, 3261, 13, 220, 198, 12, 1587, 123, 39182, 4533, 390, 2744, 260, 418, 30, 220, 198, 12, 4689, 3326, 47984, 1658, 8358, 264, 8836, 11, 627, 18082, 555, 279, 25634, 443, 73, 418, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [35]:
print("Tamaño del dataset de entrenamiento:", len(dataset["train"]))
print("Tamaño del dataset de prueba:", len(dataset["test"]))

Tamaño del dataset de entrenamiento: 1935
Tamaño del dataset de prueba: 484


In [36]:
print(dataset["train"][0])  # Imprime el primer ejemplo para ver la estructura
print(dataset["test"][0])   # Imprime el primer ejemplo para ver la estructura

{'text': '- Dígame cuál es su dirección para mandarle los documentos. \n- Calle Bosque, 31. \n- ¿Apartado de Correos? \n- La verdad es que sí, queda un poco lejos.', 'funny': 3}
{'text': '- Tío, has engórdado un montón, ¿por qué?\n- Por no discutir\n- ¡Imposible, no puede ser!\n- Vale, pues será por otra cosa', 'funny': 1}


In [45]:
print(tokenized_dataset["train"].features) 
# checking features

print(tokenized_dataset['train'][0])

print(tokenized_dataset['train'][0]['input_ids'])
print(tokenized_dataset['train'][0]['labels'])

{'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}
{'text': '- Dígame cuál es su dirección para mandarle los documentos. \n- Calle Bosque, 31. \n- ¿Apartado de Correos? \n- La verdad es que sí, queda un poco lejos.', 'input_ids': [12, 360, 8836, 6057, 18912, 6557, 75, 1658, 424, 19958, 35764, 18840, 31215, 6855, 283, 293, 22346, 3188, 418, 13, 220, 198, 12, 2199, 293, 14548, 4188, 11, 3261, 13, 220, 198, 12, 1587, 123, 39182, 4533, 390, 2744, 260, 418, 30, 220, 198, 12, 4689, 3326, 47984, 1658, 8358, 264, 8836, 11, 627, 18082, 555, 279, 25634, 443, 73, 418, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [47]:
# Extrae todos los valores de 'labels'
labels = tokenized_dataset['train'][:]["labels"]

# Encuentra los valores únicos en la columna 'labels'
unique_labels = set(labels)
print("Valores únicos en 'labels':", unique_labels)

Valores únicos en 'labels': {0, 1, 2}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')
# model = AutoModelForCausalLM.from_pretrained( "gpt2")
    
print (model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [48]:
# Configuring Lora
import peft
from peft import LoraConfig, get_peft_model, PeftModel

config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        fan_in_fan_out = True,
        # tune the embedding layer and prediction head
        modules_to_save=["wte", "lm_head"]
    )
#Create the PEFT model.

peft_model = get_peft_model(model, config)

#Conglar los parámetros
for param in peft_model.parameters():
    param.requires_grad = False
    
# Habilitar el entrenamiento solo para los parámetros del lm_head
for param in peft_model.lm_head.parameters():
    param.requires_grad = True    


In [49]:
print(peft_model.print_trainable_parameters())

trainable params: 77,194,752 || all params: 202,814,208 || trainable%: 38.06180679412756
None


In [50]:
print (peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): ModulesToSaveWrapper(
          (original_module): Embedding(50257, 768)
          (modules_to_save): ModuleDict(
            (default): Embedding(50257, 768)
          )
        )
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32,

In [51]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

In [17]:
#cheching
print(tokenized_dataset['train'][10]['input_ids'])   
print(tokenized_dataset['train'][10]['labels'])   

[23041, 555, 64, 279, 533, 6592, 11, 331, 443, 17963, 1288, 289, 2381, 260, 257, 8591, 285, 23577, 263, 25, 198, 12, 1879, 72, 31329, 11, 37062, 555, 64, 6340, 397, 430, 369, 716, 273, 13, 220, 198, 12, 3001, 9863, 3528, 52, 2885, 1581, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 502

In [20]:
print(tokenized_dataset["train"].features) 
# checking features


{'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}


In [53]:
from transformers import TrainingArguments, Trainer, TrainerCallback, pipeline, DataCollatorWithPadding


training_args = TrainingArguments(
    output_dir="./data/spam_not_spam",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,    
    num_train_epochs=1,
#    fp16=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay = 0.01,
 #   save_steps=10_000,
 #   save_total_limit=2,
 #   gradient_checkpointing=True,
    load_best_model_at_end=True
    )


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer) 
)

eval_results = trainer.evaluate()
print(f"Initial Evaluation Results: {eval_results}")





OutOfMemoryError: CUDA out of memory. Tried to allocate 3.06 GiB (GPU 0; 14.57 GiB total capacity; 13.45 GiB already allocated; 476.75 MiB free; 13.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.